In [1]:
import os

In [5]:
os.chdir("C:/Users/shin jaeik/프롬프트 과정")

In [11]:
os.getcwd()

'C:\\Users\\shin jaeik\\프롬프트 과정'

In [8]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [9]:
%%writefile module/wedding_prompt.py

import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA
import requests
import base64
from PIL import Image
import io

# API 키 설정
openai_api_key = st.secrets["OPENAI_API_KEY"]
stability_api_key = st.secrets["STABILITY_API_KEY"]

# PDF 파일 경로 설정
PDF_PATH = "data/제목 없는 문서.pdf"

# 체형 분류 함수
def classify_body_type(measurements):
    shoulder_hip_ratio = measurements['shoulder_width'] / measurements['hip_width']
    waist_hip_ratio = measurements['waist_circumference'] / measurements['hip_circumference']
    chest_waist_ratio = measurements['chest_circumference'] / measurements['waist_circumference']
    hip_waist_ratio = measurements['hip_circumference'] / measurements['waist_circumference']
    body_volume_index = (measurements['chest_circumference'] + measurements['waist_circumference'] + measurements['hip_circumference']) / 3

    if 0.95 < shoulder_hip_ratio < 1.05 and 0.95 < waist_hip_ratio < 1.05:
        return "rectangular body type", "직사각형형"
    elif 0.95 < chest_waist_ratio < 1.05 and measurements['chest_circumference'] > measurements['hip_circumference']:
        return "curvy body type", "곡선형"
    elif body_volume_index < 85:
        return "slim body type", "마른 체형"
    elif shoulder_hip_ratio > 1.1 and measurements['shoulder_width'] > measurements['hip_width']:
        return "broad-shouldered body type", "사각형 어깨형"
    elif shoulder_hip_ratio > 1.1 and measurements['chest_circumference'] > measurements['hip_circumference']:
        return "top-heavy body type", "상체 발달형"
    elif hip_waist_ratio > 1.1 and measurements['hip_circumference'] > measurements['chest_circumference']:
        return "pear-shaped body type", "하체 발달형"
    elif chest_waist_ratio > 1.1 and hip_waist_ratio > 1.1:
        return "hourglass body type", "모래시계형"
    elif hip_waist_ratio > 1.1 and measurements['hip_width'] < measurements['shoulder_width']:
        return "empire body type", "엠파이어형"
    elif hip_waist_ratio > 1.05 and measurements['hip_circumference'] > measurements['chest_circumference'] and body_volume_index < 90:
        return "slim pear body type", "슬림 하체형"
    else:
        return "balanced body type", "균형 잡힌 균형형"

# 웨딩드레스 추천 생성 함수
def get_wedding_recommendations(body_type_kr):
    recommendations = {
        "직사각형형": "A라인 드레스, 허리 강조, 레이어드 디테일 추가",
        "곡선형": "부드러운 머메이드 라인 드레스",
        "마른 체형": "볼륨감 있는 볼 가운 스타일",
        "사각형 어깨형": "V넥 라인과 A라인 스커트",
        "상체 발달형": "V넥 또는 엠파이어 드레스",
        "하체 발달형": "풀 A라인 스커트와 상체 디테일 강조",
        "모래시계형": "피트 앤 플레어 스타일",
        "엠파이어형": "하이웨이스트 라인과 쉬폰 소재 드레스",
        "슬림 하체형": "슬림한 시스 라인 드레스",
        "균형 잡힌 균형형": "클래식한 A라인 드레스"
    }
    return recommendations.get(body_type_kr, "클래식한 스타일")

# Stability AI 이미지 생성 함수
def generate_stability_image(body_type_en, stability_api_key):
    prompt = f"professional photo of a Korean bride with {body_type_en} wearing an elegant wedding dress, 8k, detailed"
    url = "https://api.stability.ai/v1/generation/stable-diffusion-xl-1024-v1-0/text-to-image"
    headers = {"Authorization": f"Bearer {stability_api_key}"}
    body = {
        "steps": 40,
        "width": 1024,
        "height": 1024,
        "cfg_scale": 7,
        "samples": 1,
        "text_prompts": [{"text": prompt, "weight": 1}]
    }

    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        image_data = base64.b64decode(response.json()["artifacts"][0]["base64"])
        return Image.open(io.BytesIO(image_data))
    else:
        return None

# PDF 로드 및 벡터 DB 설정
@st.cache_resource
def initialize_vectordb():
    try:
        loader = PyPDFLoader(PDF_PATH)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=60)
        splits = text_splitter.split_documents(documents)
        embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
        vectordb = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./VectorStores")
        return vectordb
    except Exception as e:
        st.error(f"PDF 로드 중 오류 발생: {str(e)}")
        return None

# RetrievalQA 설정
def setup_qa_chain(vectordb):
    chat_model = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})
    qa_chain = RetrievalQA.from_chain_type(
        llm=chat_model,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        verbose=True
    )
    return qa_chain

# Streamlit 인터페이스
st.title("웨딩드레스 추천 시스템")
st.write("체형 분석을 통한 맞춤형 웨딩드레스 추천")

# 사용자 입력 받기
col1, col2 = st.columns(2)
with col1:
    shoulder_width = st.number_input("어깨너비 (cm)", min_value=0.0, step=0.1)
    chest_circumference = st.number_input("가슴둘레 (cm)", min_value=0.0, step=0.1)
    waist_circumference = st.number_input("허리둘레 (cm)", min_value=0.0, step=0.1)
with col2:
    waist_width = st.number_input("허리너비 (cm)", min_value=0.0, step=0.1)
    hip_circumference = st.number_input("엉덩이둘레 (cm)", min_value=0.0, step=0.1)
    hip_width = st.number_input("엉덩이너비 (cm)", min_value=0.0, step=0.1)

if st.button("체형 분석 및 드레스 추천"):
    vectordb = initialize_vectordb()
    if vectordb is None:
        st.error("벡터 DB 초기화 실패")
    else:
        qa_chain = setup_qa_chain(vectordb)
        
        # 체형 분류 및 추천
        measurements = {
            'shoulder_width': shoulder_width,
            'chest_circumference': chest_circumference,
            'waist_circumference': waist_circumference,
            'waist_width': waist_width,
            'hip_circumference': hip_circumference,
            'hip_width': hip_width
        }
        body_type_en, body_type_kr = classify_body_type(measurements)
        recommendations = get_wedding_recommendations(body_type_kr)
        
        # 체형 분석 결과 출력
        st.write(f"### 체형 분석 결과: {body_type_kr}")
        st.write(f"추천 웨딩드레스 스타일: {recommendations}")
        
        # QA 시스템 실행
        analysis_prompt = f"체형 분석 결과 {body_type_kr}입니다. 이 체형에 가장 잘 어울리는 웨딩드레스 스타일과 그 이유를 설명해주세요."
        result = qa_chain({"query": analysis_prompt})
        st.write("### AI 기반 추가 추천사항:")
        st.write(result['result'])
        
        # Stability AI로 웨딩드레스 이미지 생성
        with st.spinner("웨딩드레스 이미지 생성 중..."):
            image = generate_stability_image(body_type_en, stability_api_key)
            if image:
                st.image(image, caption="추천 웨딩드레스 이미지")
            else:
                st.error("이미지 생성에 실패했습니다.")

if st.button("초기화"):
    st.experimental_rerun()

Overwriting module/wedding_prompt.py


In [12]:
ls data/

����ġ�� Ʋ���ϴ� - "".


In [14]:
mkdir -p ./VectorStores

���� ������ �ùٸ��� �ʽ��ϴ�.
